In [1]:
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

In [18]:
import os
from datetime import datetime
from pathlib import Path

import pandas as pd
import numpy as np

import dask
from dask import dataframe as dd
from dask import array as da
from dask_sql import Context


data_path = Path(os.getcwd()).parent/"data"
src_path = Path(os.getcwd()).parent/"src"
c = Context()

In [19]:
pdf = pd.read_pickle(data_path/"sample_cust.pkl")

In [21]:
pdf.columns

Index(['cc_cst_real', 'uniq_key', 'net_interest_rec', 'mftp_total',
       'annual_fee', 'interchange_fee', 'merchant_discount_fee',
       'cash_adv_fee', 'gain_and_loss', 'collection_fee', 'fee_waived_off',
       'fee_paid', 'debt_collection_exp', 'fee_paid_visa', 'membership_fee',
       'issuer_license_fee', 'total_issuer_domestic', 'total_issuer_inter',
       'total_account_card_fee', 'total_concierge_fee', 'net_total_income',
       'expense', 'marketing_exp', 'cashback', 'loyalty_point_redem',
       'activity_based_costing_expense', 'specific_business_tax', 'fraud_loss',
       'credit_limit_card', 'credit_limit_cust', 'credit_utilization',
       'fund_rev', 'fee_rev', 'var_cost', 'fix_cost', 'profit', 'opn_dt',
       'card_status', 'past_due_amt', 'stmt_amt', 'mth_end_bal',
       'crn_pymt_due_amt', 'util', 'cr_lmt_amt', 'mnth_cd', 'MOB', 'behv',
       'pymt_amt', 'dlq_bck', 'ews', 'b_scor', 'cc_cst_first_num'],
      dtype='object')

## Use dtype category with order to aggregate customer

In [5]:
pdf.loc[:, ["cc_cst_real", "uniq_key", "mnth_cd", "card_status", "b_scor", "behv", "dlq_bck", "ews"]]

,cc_cst_real,uniq_key,mnth_cd,card_status,b_scor,behv,dlq_bck,ews
36220,1000000000000230,3228730,202301,Valid,2,Transactor,normal,11
36780,1000000000000230,3228730,202302,Valid,2,Transactor,normal,11
38990,1000000000000230,3228730,202303,Valid,2,Transactor,normal,11
36421,1000000000000230,3228730,202304,Valid,2,Transactor,normal,11
38189,1000000000000230,3228730,202305,Valid,2,Transactor,normal,11
86291,1000000042237360,1744875,202306,Valid,11,Inactive,normal,<NA>
58695,1000000045386891,2526582,202306,Invalid,11,Inactive,90 days up,<NA>
123106,1000000045386891,4281444,202306,Invalid,11,Inactive,90 days up,<NA>
17998,1000000042237360,3055249,202306,Invalid,11,Inactive,normal,<NA>
31708,1000000042237360,2586199,202306,Invalid,11,Inactive,normal,<NA>


In [6]:
ddf = dd.from_pandas(pdf)

In [7]:
ddf["behv"].value_counts().compute()

behv
Inactive       8
Revolver       2
Transactor    12
Name: count, dtype: int64[pyarrow]

In [8]:
ddf["ews"].value_counts().compute()

ews
11    12
Name: count, dtype: int64[pyarrow]

Convert to know category with .categorize()

In [9]:
ddf = (ddf
       .fillna({"card_status":"NA", "behv":"NA", "ews":"NA", "dql_bck":"NA", "b_scor":"NA"})
       .assign(cat_card_status = lambda x : x["card_status"])
       .assign(cat_behv = lambda x : x["behv"])
       # .assign(cat_ews = lambda x : x["ews"])
       .assign(cat_dlq = lambda x : x["dlq_bck"])
       .assign(cat_bscor = lambda x : x["b_scor"])
       .categorize(columns=["cat_card_status", "cat_behv", "cat_dlq", "cat_bscor"])
)

In [69]:
ddf["cat_card_status"].cat.known

True

In [70]:
ddf["cat_behv"].cat.known

True

In [10]:
ddf["cat_bscor"].cat.known

True

In [11]:
ddf["cat_dlq"].cat.known

True

Add order to category with .cat.set_categories([],order=True)

In [12]:
status_order = ["NA", "Invalid", "Valid"]
behv_order = ["NA", "Inactive", "Transactor", "Revolver"]
# ews_order = [4,3,2,1]
dlq_order = ["NA", "normal", "1-30", "31-60", "61-90", "90 days up"]
bscor_order = ["NA", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11"]

ddf = (ddf
 .assign(cat_card_status = lambda x : x["cat_card_status"].cat.set_categories(status_order, ordered=True))
 .assign(cat_behv = lambda x : x["cat_behv"].cat.set_categories(behv_order, ordered=True))
#  .assign(cat_ews = lambda x : x["cat_ews"].cat.set_categories(ews_order, ordered=True))
 .assign(cat_dlq = lambda x : x["cat_dlq"].cat.set_categories(dlq_order, ordered=True))
 .assign(cat_bscor = lambda x : x["cat_bscor"].cat.set_categories(bscor_order, ordered=True))
)

Aggregate top order with .groupby().max()  
**all the category dtype must be in the agg**

In [13]:
ddf.groupby(["cc_cst_real", "mnth_cd"]).agg({"uniq_key":"count","cat_behv":"max","cat_card_status":"max", "cat_dlq":"max", "cat_bscor":"max"}).compute()

uniq_key    cat_behv cat_card_status     cat_dlq  \
cc_cst_real      mnth_cd                                                     
1000000042237360 202306          6    Revolver           Valid       61-90   
1000000000000230 202301          1  Transactor           Valid      normal   
                 202304          1  Transactor           Valid      normal   
                 202302          1  Transactor           Valid      normal   
                 202306          1  Transactor           Valid      normal   
                 202308          1  Transactor           Valid      normal   
                 202305          1  Transactor           Valid      normal   
                 202307          1  Transactor           Valid      normal   
                 202303          1  Transactor           Valid      normal   
                 202310          1  Transactor           Valid      normal   
                 202311          1  Transactor           Valid      normal   
                 202309          1  Transactor           Valid      normal   
                 202312          1  Transactor           Valid      normal   
1000000045386891 202306          2    Inactive         Invalid  90 days up   
                 202307          2    Inactive         Invalid  90 days up   

                         cat_bscor  
cc_cst_real      mnth_cd            
1000000042237360 202306         11  
1000000000000230 202301          2  
                 202304          2  
                 202302          2  
                 202306          2  
                 202308          2  
                 202305          2  
                 202307          2  
                 202303          2  
                 202310          2  
                 202311          2  
                 202309          2  
                 202312          2  
1000000045386891 202306         11  
                 202307         11

Category dtype, with `NaN` will be max/min

In [14]:
sel_col = ["cc_cst_real", "uniq_key", "mnth_cd", "behv", "card_status", "dlq_bck", "b_scor"]
pdf.query("cc_cst_real == 1000000042237360").loc[:,sel_col]

,cc_cst_real,uniq_key,mnth_cd,behv,card_status,dlq_bck,b_scor
86291,1000000042237360,1744875,202306,Inactive,Valid,normal,11
17998,1000000042237360,3055249,202306,Inactive,Invalid,normal,11
31708,1000000042237360,2586199,202306,Inactive,Invalid,normal,11
127007,1000000042237360,3565641,202306,Revolver,Valid,31-60,11
75692,1000000042237360,1880373,202306,Inactive,Invalid,normal,11
90171,1000000042237360,4267734,202306,Revolver,Valid,61-90,11


In [15]:
pdf.query("cc_cst_real == 1000000000000230").loc[:,sel_col]

,cc_cst_real,uniq_key,mnth_cd,behv,card_status,dlq_bck,b_scor
36220,1000000000000230,3228730,202301,Transactor,Valid,normal,2
36780,1000000000000230,3228730,202302,Transactor,Valid,normal,2
38990,1000000000000230,3228730,202303,Transactor,Valid,normal,2
36421,1000000000000230,3228730,202304,Transactor,Valid,normal,2
38189,1000000000000230,3228730,202305,Transactor,Valid,normal,2
37728,1000000000000230,3228730,202306,Transactor,Valid,normal,2
38405,1000000000000230,3228730,202307,Transactor,Valid,normal,2
38092,1000000000000230,3228730,202308,Transactor,Valid,normal,2
39840,1000000000000230,3228730,202309,Transactor,Valid,normal,2
39114,1000000000000230,3228730,202310,Transactor,Valid,normal,2


In [16]:
pdf.query("cc_cst_real == 1000000045386891").loc[:,sel_col]

,cc_cst_real,uniq_key,mnth_cd,behv,card_status,dlq_bck,b_scor
58695,1000000045386891,2526582,202306,Inactive,Invalid,90 days up,11
123106,1000000045386891,4281444,202306,Inactive,Invalid,90 days up,11
59345,1000000045386891,2526582,202307,Inactive,Invalid,90 days up,11
124320,1000000045386891,4281444,202307,Inactive,Invalid,90 days up,11


If 

In [19]:
cat_agg_ddf = ddf.groupby(["cc_cst_real", "mnth_cd"]).agg({"uniq_key":"count","cat_behv":"max","cat_card_status":"max", "cat_dlq":"max", "cat_bscor":"max"})

In [20]:
cat_agg_ddf.dtypes

uniq_key              int64
cat_behv           category
cat_card_status    category
cat_dlq            category
cat_bscor          category
dtype: object

Convert category to string type and save

In [21]:
# This command will error, since the cat_ews not in output
cat_agg_ddf.reset_index().astype({"cat_behv":"string", "cat_card_status":"string", "cat_dlq":"string", "cat_bscor":"string"}).to_parquet(data_path/"cat_agg_ddf.parquet", overwrite=True)

In [22]:
cat_ddf = dd.read_parquet(data_path/"cat_agg_ddf.parquet")

In [23]:
cat_ddf.dtypes

cc_cst_real                 int64
mnth_cd                     int32
uniq_key                    int64
cat_behv           string[python]
cat_card_status    string[python]
cat_dlq            string[python]
cat_bscor          string[python]
dtype: object

In [29]:
bscor_grade = dd.read_csv(src_path/"bscor_grade_mapper.csv", dtype={"cat_bscor":"string"})

In [31]:
cat_ddf.merge(bscor_grade, on="cat_bscor", how="left").compute()

,cc_cst_real,mnth_cd,uniq_key,cat_behv,cat_card_status,cat_dlq,cat_bscor,b_grade
0,1000000042237360,202306,6,Revolver,Valid,61-90,11,VH
1,1000000000000230,202301,1,Transactor,Valid,normal,2,VL
2,1000000000000230,202304,1,Transactor,Valid,normal,2,VL
3,1000000000000230,202302,1,Transactor,Valid,normal,2,VL
4,1000000000000230,202306,1,Transactor,Valid,normal,2,VL
5,1000000000000230,202308,1,Transactor,Valid,normal,2,VL
6,1000000000000230,202305,1,Transactor,Valid,normal,2,VL
7,1000000000000230,202307,1,Transactor,Valid,normal,2,VL
8,1000000000000230,202303,1,Transactor,Valid,normal,2,VL
9,1000000000000230,202310,1,Transactor,Valid,normal,2,VL


## Use Dask .map to mapping value in series

In [124]:
pdf = pd.read_pickle(data_path/"sample_cust.pkl")
ddf = dd.from_pandas(pdf)

In [97]:
mapped = (ddf
 .assign(corp_flag = lambda x : (x["cc_cst_real"]/1e15).astype("int"))
 .assign(card_typ = lambda x : x["corp_flag"].map({1:"comm", 2:"corp"}, meta=("corp_flag", "string")))
 .assign(card_typ = lambda x : x["corp_flag"].map({2:"corp"}, meta=("corp_flag", "string"))) # Test if not complete list
)

In [98]:
mapped.compute().loc[: ,["cc_cst_real", "corp_flag", "card_typ"]]

,cc_cst_real,corp_flag,card_typ
17998,1000000042237360,1,NaN
31708,1000000042237360,1,NaN
36220,1000000000000230,1,NaN
36421,1000000000000230,1,NaN
36780,1000000000000230,1,NaN
37728,1000000000000230,1,NaN
38092,1000000000000230,1,NaN
38189,1000000000000230,1,NaN
38405,1000000000000230,1,NaN
38990,1000000000000230,1,NaN


## Use Dask Array to replicate np.select
syntax
```
dff["new_col_name"] = \
   da.where(cond#1, value if cond#1 true, 
    da.where(cond#2, value if cond#2 true,
        da.where(cond#3, value if cond#3 true, default value)
            )
           )
```
Same as `case-when`, have short circuit property = if satisfy the condition will not further check the lower rank condition

In [5]:
pdf = pd.read_pickle(data_path/"sample_cust.pkl")
ddf = dd.from_pandas(pdf)
ddf.compute()

x = ddf["net_interest_rec"].to_dask_array(lengths=True)
y = ddf["behv"].to_dask_array(lengths=True)

/home/danny/miniconda3/envs/dask/lib/python3.12/site-packages/dask/dataframe/core.py:3930: UserWarning: Dask currently has limited support for converting pandas extension dtypes to arrays. Converting string to object dtype.
  warnings.warn(


In [46]:
ddf['TEST_VAR'] = \
    da.where(x == 0, 'THIS',
        da.where(y == "Revolver", 'NOT THIS',
            da.where((x > 0) & (y != "Inactive"), 'THAT', 'NONE')
                )
            )
ddf[["net_interest_rec", "behv", "TEST_VAR"]].compute()

,net_interest_rec,behv,TEST_VAR
17998,0.00,Inactive,THIS
31708,0.00,Inactive,THIS
36220,0.00,Transactor,THIS
36421,0.00,Transactor,THIS
36780,0.00,Transactor,THIS
37728,0.00,Transactor,THIS
38092,0.00,Transactor,THIS
38189,0.00,Transactor,THIS
38405,0.00,Transactor,THIS
38990,0.00,Transactor,THIS


## Resample
To create groupe by at frequency at monthly

In [33]:
pdf["mnth_dt"] = pd.to_datetime(pdf["mnth_cd"],format="%Y%m")
pdf.loc[:, ["cc_cst_real","uniq_key","mnth_cd","mnth_dt","profit"]]
timed_idx = pdf.set_index("mnth_dt") #.resample("MS").sum()
timed_idx.loc[:, ["cc_cst_real","uniq_key","mnth_cd","profit"]]

,cc_cst_real,uniq_key,mnth_cd,profit
mnth_dt,,,,
2023-01-01,1000000000000230,3228730,202301,-81.551505
2023-02-01,1000000000000230,3228730,202302,-80.173696
2023-03-01,1000000000000230,3228730,202303,-84.272939
2023-04-01,1000000000000230,3228730,202304,-84.647109
2023-05-01,1000000000000230,3228730,202305,-77.076268
2023-06-01,1000000042237360,1744875,202306,-107.771390
2023-06-01,1000000045386891,2526582,202306,0.000000
2023-06-01,1000000045386891,4281444,202306,-10.211657
2023-06-01,1000000042237360,3055249,202306,0.000000


In [37]:
timed_idx.loc[:, ["cc_cst_real","uniq_key","mnth_cd","profit"]].groupby(["cc_cst_real"]).resample("MS").sum()

cc_cst_real  uniq_key  mnth_cd      profit
cc_cst_real      mnth_dt                                                    
1000000000000230 2023-01-01  1000000000000230   3228730   202301  -81.551505
                 2023-02-01  1000000000000230   3228730   202302  -80.173696
                 2023-03-01  1000000000000230   3228730   202303  -84.272939
                 2023-04-01  1000000000000230   3228730   202304  -84.647109
                 2023-05-01  1000000000000230   3228730   202305  -77.076268
                 2023-06-01  1000000000000230   3228730   202306 -107.771390
                 2023-07-01  1000000000000230   3228730   202307  -78.076071
                 2023-08-01  1000000000000230   3228730   202308  -70.941228
                 2023-09-01  1000000000000230   3228730   202309  -79.568657
                 2023-10-01  1000000000000230   3228730   202310  -88.735632
                 2023-11-01  1000000000000230   3228730   202311  -83.086975
                 2023-12-01  1000000000000230   3228730   202312 -127.898626
1000000042237360 2023-06-01  6000000253424160  17100071  1213836  464.078941
1000000045386891 2023-06-01  2000000090773782   6808026   404612  -10.211657
                 2023-07-01  2000000090773782   6808026   404614    0.000000

## Account code logic = shift 3 digits (cust*1000) + card type code (3 digits)

In [162]:
pdf = pd.read_pickle(data_path/"sample_cust.pkl")
ddf = dd.from_pandas(pdf)

mapper = (dd.from_pandas(pd.read_csv(src_path/"card_type_mapper.csv"))
            .astype({"card_tp":"string", "issur_card_lvl2":"string"})
)

In [163]:
mapper.persist()

,card_tp,issur_card_lvl2,card_typ_cd,card_lv_cd,card_typ_lv_cd
npartitions=1,,,,,
0,string,string,int64,int64,int64
22,...,...,...,...,...


**Maximum size for int64 = cc_cst_real*1000**  
card type code have 3 digits, then shift cc_cst_real * 1000

In [164]:
ddf["act_no"] = ddf["cc_cst_real"]*1000+244
ddf.loc[:, ["cc_cst_real", "act_no"]].compute()

,cc_cst_real,act_no
17998,1000000042237360,1000000042237360244
31708,1000000042237360,1000000042237360244
36220,1000000000000230,1000000000000230244
36421,1000000000000230,1000000000000230244
36780,1000000000000230,1000000000000230244
37728,1000000000000230,1000000000000230244
38092,1000000000000230,1000000000000230244
38189,1000000000000230,1000000000000230244
38405,1000000000000230,1000000000000230244
38990,1000000000000230,1000000000000230244


In [165]:
ddf["card_typ_lv_cd"] = (ddf["act_no"].mod(1000)).astype("int")
ddf.loc[:, ["cc_cst_real", "act_no", "card_typ_lv_cd"]].compute()

,cc_cst_real,act_no,card_typ_lv_cd
17998,1000000042237360,1000000042237360244,244
31708,1000000042237360,1000000042237360244,244
36220,1000000000000230,1000000000000230244,244
36421,1000000000000230,1000000000000230244,244
36780,1000000000000230,1000000000000230244,244
37728,1000000000000230,1000000000000230244,244
38092,1000000000000230,1000000000000230244,244
38189,1000000000000230,1000000000000230244,244
38405,1000000000000230,1000000000000230244,244
38990,1000000000000230,1000000000000230244,244


In [168]:
ddf.merge(mapper.drop(["card_typ_cd", "card_lv_cd"], axis=1), on="card_typ_lv_cd", how="left").compute()

,cc_cst_real,uniq_key,net_interest_rec,mftp_total,annual_fee,interchange_fee,merchant_discount_fee,cash_adv_fee,gain_and_loss,collection_fee,...,behv,pymt_amt,dlq_bck,ews,b_scor,cc_cst_first_num,act_no,card_typ_lv_cd,card_tp,issur_card_lvl2
0,1000000042237360,3055249,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,Inactive,0.00,normal,<NA>,11,1,1000000042237360244,244,Wisdom,SIGNATURE
1,1000000042237360,2586199,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,Inactive,0.00,normal,<NA>,11,1,1000000042237360244,244,Wisdom,SIGNATURE
2,1000000000000230,3228730,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,Transactor,0.00,normal,11,2,1,1000000000000230244,244,Wisdom,SIGNATURE
3,1000000000000230,3228730,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,Transactor,0.00,normal,11,2,1,1000000000000230244,244,Wisdom,SIGNATURE
4,1000000000000230,3228730,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,Transactor,0.00,normal,11,2,1,1000000000000230244,244,Wisdom,SIGNATURE
5,1000000000000230,3228730,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,Transactor,0.00,normal,11,2,1,1000000000000230244,244,Wisdom,SIGNATURE
6,1000000000000230,3228730,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,Transactor,0.00,normal,11,2,1,1000000000000230244,244,Wisdom,SIGNATURE
7,1000000000000230,3228730,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,Transactor,0.00,normal,11,2,1,1000000000000230244,244,Wisdom,SIGNATURE
8,1000000000000230,3228730,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,Transactor,0.00,normal,11,2,1,1000000000000230244,244,Wisdom,SIGNATURE
9,1000000000000230,3228730,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,Transactor,0.00,normal,11,2,1,1000000000000230244,244,Wisdom,SIGNATURE


## Collect as list/set

In [163]:
pdf = pd.read_pickle(data_path/"sample_cust.pkl")
ddf = dd.from_pandas(pdf)

In [177]:
pdf.groupby("cc_cst_real")["behv"].agg(list).reset_index()

,cc_cst_real,behv
0,1000000000000230,"[Transactor, Transactor, Transactor, Transacto..."
1,1000000042237360,"[Inactive, Inactive, Inactive, Revolver, Inact..."
2,1000000045386891,"[Inactive, Inactive, Inactive, Inactive]"


In [171]:
list_agg = ddf.groupby("cc_cst_real")["behv"].agg(list)

In [173]:
list_agg.compute().reset_index()

,cc_cst_real,behv
0,1000000042237360,"[Inactive, Inactive, Inactive, Inactive, Revol..."
1,1000000000000230,"[Transactor, Transactor, Transactor, Transacto..."
2,1000000045386891,"[Inactive, Inactive, Inactive, Inactive]"


In [180]:
pdf.groupby("cc_cst_real")["behv"].agg(set_behv= lambda x : set(x))

,set_behv
cc_cst_real,
1000000000000230,{Transactor}
1000000042237360,"{Revolver, Inactive}"
1000000045386891,{Inactive}


In [186]:
set_agg = ddf.groupby("cc_cst_real")["behv"].agg(set_behv=set)

ValueError: unknown aggregate set

## GroupBy, nlargest

In [187]:
pdf = pd.read_pickle(data_path/"sample_cust.pkl")
ddf = dd.from_pandas(pdf)

In [210]:
pdf.groupby("cc_cst_real").nth(3).loc[:, ["cc_cst_real", "mnth_cd", "uniq_key"]]

,cc_cst_real,mnth_cd,uniq_key
36421,1000000000000230,202304,3228730
127007,1000000042237360,202306,3565641
124320,1000000045386891,202307,4281444


In [207]:
pdf.query("cc_cst_real == 1000000045386891").loc[:, ["cc_cst_real", "mnth_cd", "uniq_key"]]

,cc_cst_real,mnth_cd,uniq_key
58695,1000000045386891,202306,2526582
123106,1000000045386891,202306,4281444
59345,1000000045386891,202307,2526582
124320,1000000045386891,202307,4281444


In [212]:
ddf.groupby("cc_cst_real")["mnth_cd"].nth(3).loc[:, ["cc_cst_real", "mnth_cd", "uniq_key"]]

AttributeError: 'Column not found: nth'